In [ ]:
# ===================================================
# YOLOv8 Colab 一鍵訓練 + 影片物件偵測
# ===================================================

import os
import cv2
import numpy as np
from google.colab import drive
from IPython.display import HTML, display

# -----------------------------
# 1️⃣ 安裝套件（使用 %pip 避免 subprocess 問題）
# -----------------------------
%pip install -q --upgrade numpy==1.24.4
%pip install -q --upgrade ultralytics opencv-python
%pip install -q --upgrade tensorflow==2.19.0

# -----------------------------
# 2️⃣ 掛載 Google Drive
# -----------------------------
drive.mount('/content/drive', force_remount=True)

PROJECT_DIR = "/content/drive/MyDrive/20251226AI/yolo_project"
DATA_DIR = "/content/drive/MyDrive/20251226AI/yolo_data"

# -----------------------------
# 3️⃣ 建立資料夾結構
# -----------------------------
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "labels/val"), exist_ok=True)

# -----------------------------
# 4️⃣ 建立範例 data.yaml（如果不存在）
# -----------------------------
yaml_path = os.path.join(DATA_DIR, "data.yaml")
if not os.path.exists(yaml_path):
    with open(yaml_path, "w") as f:
        f.write(f"""# YOLOv8 dataset config
path: {DATA_DIR}
train: images/train
val: images/val
names:
  0: class0
  1: class1
""")
    print(f"✅ 已建立空的 data.yaml: {yaml_path}")
else:
    print(f"✅ data.yaml 已存在: {yaml_path}")

# -----------------------------
# 5️⃣ YOLOv8 訓練
# -----------------------------
from ultralytics import YOLO

# 將模型下載到本地 /content 避免 Drive 斷線問題
MODEL_PATH = "/content/yolov8n.pt"
model = YOLO(MODEL_PATH)

# 執行訓練（即使資料夾空也會跳過）
model.train(
    data=yaml_path,
    epochs=1,         # 範例測試用，正式可改大
    imgsz=640,
    batch=16,
    name="yolo_finetune",
    project=PROJECT_DIR
)

# 載入訓練完成權重，如果不存在就用預訓練模型
TRAINED_MODEL_PATH = f"{PROJECT_DIR}/yolo_finetune/weights/best.pt"
if os.path.exists(TRAINED_MODEL_PATH):
    model = YOLO(TRAINED_MODEL_PATH)
    print(f"✅ 訓練權重載入成功: {TRAINED_MODEL_PATH}")
else:
    print("⚠️ 訓練權重不存在，使用預訓練模型進行影片偵測")

# -----------------------------
# 6️⃣ 影片物件偵測
# -----------------------------
video_path = "/content/sample_video.avi"
output_path = f"{PROJECT_DIR}/result_detect.mp4"

# 下載範例影片（如果不存在）
if not os.path.exists(video_path):
    !wget -O /content/sample_video.avi https://github.com/opencv/opencv/raw/master/samples/data/vtest.avi

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

COLORS = {}
def get_color(class_id):
    if class_id not in COLORS:
        np.random.seed(class_id + 10)
        COLORS[class_id] = tuple(int(x) for x in np.random.randint(50, 255, 3))
    return COLORS[class_id]

for result in model.track(source=video_path, stream=True, tracker="botsort.yaml"):
    frame = result.orig_img.copy()
    if result.boxes is not None:
        for box in result.boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            color = get_color(cls)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    out.write(frame)

cap.release()
out.release()

# -----------------------------
# 7️⃣ 顯示結果影片
# -----------------------------
display(HTML(f"""<video width='640' height='480' controls>
<source src='{output_path}' type='video/mp4'>
</video>"""))

# -----------------------------
# 8️⃣ 列出資料夾結構（Python 替代 tree）
# -----------------------------
for root, dirs, files in os.walk(DATA_DIR):
    level = root.replace(DATA_DIR, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Mounted at /content/drive
✅ data.yaml 已存在: /content/drive/MyDrive/20251226AI/yolo_data/data.yaml
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmi

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all          2          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels

1 epochs completed in 0.001 hours.
Optimizer stripped from /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune5/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune5/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune5/weights/best.pt...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 31.3it/s 0.0s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all          2          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.3ms preprocess, 6.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune5
⚠️ 訓練權重不存在，使用預訓練模型進行影片偵測
--2025-12-29 02:05:46--  https://github.com/opencv/opencv/raw/master/samples/data/vtest.avi
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opencv/opencv/master/samples/data/vtest.avi [following]
--2025-12-29 02:05:47--  https://raw.githubusercontent.com/opencv/opencv/master/samples/data/vtest.avi
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

yolo_data/
    data.yaml.gdoc
    data.yaml
    images/
        train/
            people.jpg
            car1.jpg
        val/
            people.jpg
            car1.jpg
    labels/
        train.cache
        val.cache
        train/
            people.txt.gdoc
            car1.txt.gdoc
        val/
            people.txt.gdoc
            car1.txt.gdoc
